<a href="https://colab.research.google.com/github/Stravanni/Basi_di_dati/blob/main/03_SQL_sqlite_nested_any_in_exists.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduzione a SQL
@author: giovanni.simonini@unimore.it

## To run: 
- to run a cell: SHIFT + ENTER
  
## Schema Università

S (<u>Matr</u>,SNome,Citta,ACorso)

D(<u>CD</u>,CNome,Citta)

C(<u>CC</u>,CNome,CD)
- FOREIGN KEY (CD) REFERENCES D(CD)

E(<u>Matr,CC</u>,DATA,VOTO)
- FOREIGN KEY (Matr) REFERENCES S(Matr)
- FOREIGN KEY (CC) REFERENCES C(CC)
 

In [6]:
import pandas as pd
from sqlalchemy import create_engine

In [7]:
engine = create_engine('sqlite://', echo=False)

## Create the the tables

In [8]:
q = '''
CREATE TABLE S (
    Matr VARCHAR(45),
    SNome VARCHAR(45),
    Citta VARCHAR(45),
    ACorso INT,
    PRIMARY KEY (Matr)
);
'''
engine.execute(q)

q = '''
CREATE TABLE D(
 CD VARCHAR(45),
 CNome VARCHAR(45),
 Citta VARCHAR(45),
 PRIMARY KEY (CD)
);
'''
engine.execute(q)

q = '''
CREATE TABLE C(
 CC VARCHAR(45),
 CNome VARCHAR(45),
 CD VARCHAR(45),
 PRIMARY KEY (CC),
 FOREIGN KEY (CD) REFERENCES D(CD)
);
'''
engine.execute(q)

q = '''
CREATE TABLE E(
 Matr VARCHAR(45),
 CC VARCHAR(45),
 DATA DATE,
 VOTO INT,
 PRIMARY KEY (Matr, CC),
 FOREIGN KEY (Matr) REFERENCES S(Matr),
 FOREIGN KEY (CC) REFERENCES C(CC)
);
'''
engine.execute(q)


q = '''
INSERT INTO S (Matr, SNome, Citta, ACorso)
VALUES
('M1','Lucia Quaranta','SA',1),
('M2','Giacomo Tedesco','PA',2),
('M3','Carla Longo','MO',1),
('M4','Ugo Rossi','MO',1),
('M5','Valeria Neri','MO',2),
('M6','Giuseppe Verdi','BO',1),
('M7','Maria Rossi',null,1);
'''
engine.execute(q)


q = '''
INSERT INTO D (CD, CNome, Citta)
VALUES
('D1','Paolo Rossi','MO'),
('D2','Maria Pastore','BO'),
('D3','Paola Caboni','FI');
'''
engine.execute(q)


q = '''
INSERT INTO C (CC,CNome, CD)
VALUES
('C1','Fisica 1','D1'),
('C2','Analisi Matematica 1','D2'),
('C3','Fisica 2','D1'),
('C4','Analisi Matematica 2','D3');
'''
engine.execute(q)


q = '''
INSERT INTO E (Matr,CC,Data,Voto)
VALUES
('M1','C1','1995-06-29',24),
('M1','C2','1996-08-09',33),
('M1','C3','1996-03-12',30),
('M2','C1','1995-06-29',28),
('M2','C2','1996-07-07',24),
('M3','C2','1996-07-07',27),
('M3','C3','1996-11-11',25),
('M4','C3','1996-11-11',33),
('M6','C2','1996-01-02',28),
('M7','C1','1995-06-29',24),
('M7','C2','1996-04-11',26),
('M7','C3','1996-06-23',27);
'''
engine.execute(q)

## Query the DB

In [9]:
# TEMPLATE QUERY
q = '''
SELECT *
FROM S,C,D,E
'''

res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df.head()

,Matr,SNome,Citta,ACorso,CC,CNome,CD,CD,CNome,Citta,Matr,CC,DATA,VOTO
0,M1,Lucia Quaranta,SA,1,C1,Fisica 1,D1,D1,Paolo Rossi,MO,M1,C1,1995-06-29,24
1,M1,Lucia Quaranta,SA,1,C1,Fisica 1,D1,D1,Paolo Rossi,MO,M1,C2,1996-08-09,33
2,M1,Lucia Quaranta,SA,1,C1,Fisica 1,D1,D1,Paolo Rossi,MO,M1,C3,1996-03-12,30
3,M1,Lucia Quaranta,SA,1,C1,Fisica 1,D1,D1,Paolo Rossi,MO,M2,C1,1995-06-29,28
4,M1,Lucia Quaranta,SA,1,C1,Fisica 1,D1,D1,Paolo Rossi,MO,M2,C2,1996-07-07,24


## Nested Quereis

In [10]:
# Interrogazioni innestate

# Una interrogazione viene detta innestata o nidificata se la sua condizione
# e' formulata usando il risultato di un’altra interrogazione, 
# chiamata subquery

q = '''
SELECT *
FROM S
WHERE S.Matr IN (SELECT Matr
				 FROM E);
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,Matr,SNome,Citta,ACorso
0,M1,Lucia Quaranta,SA,1
1,M2,Giacomo Tedesco,PA,2
2,M3,Carla Longo,MO,1
3,M4,Ugo Rossi,MO,1
4,M6,Giuseppe Verdi,BO,1
5,M7,Maria Rossi,None,1


## ANY/IN/EXISTS
- Operatori quantificati: `<attr><op-rel>[ANY|ALL]<subquery`
- Operatore di set: `<attr> [NOT] IN <subquery>`
- Quantificatore esistenziale : `[NOT] EXISTS <subquery>`

In [11]:
#  TIPO [ANY/ALL]
# Per esprimere una condizione che e' vera quando 
# qualsiasi (ANY) opppure tutti (ALL) gli elementi
# restituiti dalla subquery fanno verificare la condizione
# i.e., condizione risulta TRUE
 
# ANY
# Nome degli studenti che hanno sostenuto l’esame del corso C1

q = '''
SELECT SNome 
FROM S 
WHERE Matr = ANY (SELECT Matr FROM E WHERE CC='C1');
'''

# SQLite non ha "ANY"
q = '''
SELECT SNome 
FROM S 
WHERE Matr IN (SELECT Matr FROM E WHERE CC='C1');
'''

res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,SNome
0,Lucia Quaranta
1,Giacomo Tedesco
2,Maria Rossi


In [12]:
q = '''
SELECT SNome
FROM S natural JOIN E
WHERE E.CC = 'C1';
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,SNome
0,Lucia Quaranta
1,Giacomo Tedesco
2,Maria Rossi


In [13]:
# ALL
# Studenti con anno di corso piu` basso

# SQLlite non ha "ALL"

q = '''
SELECT *
FROM S
WHERE ACorso <= ALL ( SELECT ACorso
					  FROM S);
'''

q = '''
SELECT *
FROM S
WHERE ACorso <= ( SELECT MAX(ACorso)
					  FROM S);
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,Matr,SNome,Citta,ACorso
0,M1,Lucia Quaranta,SA,1
1,M2,Giacomo Tedesco,PA,2
2,M3,Carla Longo,MO,1
3,M4,Ugo Rossi,MO,1
4,M5,Valeria Neri,MO,2
5,M6,Giuseppe Verdi,BO,1
6,M7,Maria Rossi,None,1


In [14]:
# Spesso possono esserci diversi modi di esprimere la stesa query
# es: Nome degli studenti che hanno sostenuto l’esame del corso C1
# senza subquery:
q = '''
SELECT S.SNome
FROM E join S on E.Matr=S.Matr
WHERE E.CC = 'C1';
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

# # oppure:
# q = '''
# SELECT S.SNome
# FROM S
# WHERE S.Matr in (SELECT E.Matr 
# 				 FROM E WHERE E.CC='C1');
# '''
# con.execute(q) 
# con.fetchdf()

,SNome
0,Lucia Quaranta
1,Giacomo Tedesco
2,Maria Rossi


In [15]:
# ALTRO ESEMPIO:
# Nome degli studenti che hanno sostenuto l’esame di un corso del docente D1
#/
q = '''
SELECT SNome
FROM E,S,C
WHERE E.Matr=S.Matr 
AND E.CC=C.CC
AND CD='D1';
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,SNome
0,Lucia Quaranta
1,Lucia Quaranta
2,Giacomo Tedesco
3,Carla Longo
4,Ugo Rossi
5,Maria Rossi
6,Maria Rossi


In [16]:
# che puo' essere espressa anche come:
q = '''
SELECT SNome
FROM S
WHERE S.Matr IN (SELECT E.Matr 
				 FROM E 
				 WHERE E.CC IN (SELECT C.CC 
				 				FROM C 
				 				WHERE C.CD='D1'));
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,SNome
0,Lucia Quaranta
1,Giacomo Tedesco
2,Carla Longo
3,Ugo Rossi
4,Maria Rossi


In [17]:
# Provare la precedente query partendo dalla subquery piu' interna

# Fino ad ora abbiamo considerato delle subquery dove
# la query innestata era indipendente dalla query esterna

# Se vogliamo pero' selezionare:
# Per ogni citta', il nome degli studenti con anno di corso piu` alto
q = '''
SELECT S1.Citta, S1.SNome, S1.ACorso
FROM S S1
WHERE S1.ACorso >= (SELECT MAX(S2.ACorso)
					  	FROM S S2
					  	WHERE S1.Citta=S2.Citta);
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df
# S1 e' la tupla corrente

,Citta,SNome,ACorso
0,SA,Lucia Quaranta,1
1,PA,Giacomo Tedesco,2
2,MO,Valeria Neri,2
3,BO,Giuseppe Verdi,1


### EXISTS

In [18]:
# QUALIFICATORE ESISTENZIALE:

# EXISTS

# Mettendo in relazione subquery e query estenza con una clausola
# che è vera solo se per la tupla corrente il risultato della subquery
# restituisce qualcosa (i.e., non è nullo).
# Da qui: "EXISTS", cioè ESISTE qualcosa che rende TRUE quel predicato.

# Ad esempio:
# Nome degli studenti che hanno sostenuto l’esame del corso C1:

q = '''
SELECT SNome
FROM S
WHERE EXISTS (SELECT *
			  FROM E
			  WHERE E.Matr=S.Matr
			  AND E.CC = 'C1');
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,SNome
0,Lucia Quaranta
1,Giacomo Tedesco
2,Maria Rossi


In [19]:
# Allo stesso modo, il negato della clausola EXISTS:

# NOT EXISTS (<subquery>)

# ha valore true se e solo se l’insieme di valori restituiti da <subquery> e' vuoto.

# Ad esempio:
# Nome degli studenti che non hanno sostenuto l’esame del corso C1
q = '''
SELECT SNome
FROM S
WHERE NOT EXISTS ( SELECT *
				   FROM E
				   WHERE E.Matr=S.Matr 
				   AND E.CC='C1' )
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

# Tip:
# Se state usando EXISTS senza una subquery crrelata,
# molto probabilmente state commettendo un errore:

# Perche'?
# (hint: cosa restituisce la subquery?) 

,SNome
0,Carla Longo
1,Ugo Rossi
2,Valeria Neri
3,Giuseppe Verdi


### Riduzione di query innestate

In [20]:
# Le query innestate formulate con i seguenti operatori 
# si possono ridurre a query di join equivalenti 
# (stessa risposta per ogni possibile istanza della base di dati):
# • IN
# • ANY (con qualsiasi operatore di confronto) 
# • EXISTS con subquery correlata
# 

				   
# Nome degli studenti che hanno sostenuto l’esame del corso C1

# 1. 
q = '''
SELECT SNome 
FROM S
WHERE Matr IN ( SELECT Matr
				FROM E
				WHERE CC='C1')
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

# # 2. 
# q = '''
# SELECT SNome 
# FROM S
# WHERE Matr =ANY ( SELECT Matr
# 				  FROM E
# 				  WHERE CC='C1')
# '''
# res = engine.execute(q)
# df = pd.DataFrame(res.fetchall())
# df.columns = res.keys()
# df

# 3.
q = '''
SELECT SNome 
FROM S
WHERE EXISTS ( SELECT *
			   FROM E
			   WHERE E.Matr=S.Matr
			   AND CC='C1')
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

			   
# QUERY EQUIVALENTE CON JOIN
q = '''
SELECT SNome
FROM E,S
WHERE E.Matr=S.Matr 
AND E.CC='C1'
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,SNome
0,Lucia Quaranta
1,Giacomo Tedesco
2,Maria Rossi


In [21]:
# INVECE:
 
# Le query innestate formulate con i seguenti operatori 
# non si possono ridurre:

# • NOT IN
# • ALL (con qualsiasi operatore di confronto) 
# • NOT EXISTS con subquery correlata

# Ad esempio:
# Nome degli studenti che non hanno sostenuto l’esame del corso C1

# 1. 
q = '''
SELECT SNome 
FROM S
WHERE Matr NOT IN ( SELECT Matr
	 				FROM E
					WHERE CC='C1')
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

# # 2.
# q = '''
# SELECT SNome 
# FROM S
# WHERE Matr <> ALL ( SELECT Matr
# 				  FROM E
# 				  WHERE CC='C1')
# '''
# res = engine.execute(q)
# df = pd.DataFrame(res.fetchall())
# df.columns = res.keys()
# df


# 3.
q = '''
SELECT SNome 
FROM S
WHERE NOT EXISTS ( SELECT *
			   FROM E
			   WHERE E.Matr=S.Matr
			   AND CC='C1')
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

# NON SI POSSONO ESPRIMRE CON JOIN

,SNome
0,Carla Longo
1,Ugo Rossi
2,Valeria Neri
3,Giuseppe Verdi


In [22]:
# NOTARE: la differenza con la seguente query
# ERRATA (per la consegna):
q = '''
SELECT SNome
FROM E,S
WHERE E.Matr=S.Matr 
AND E.CC <> 'C1'
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

# cosa restituisce?

,SNome
0,Lucia Quaranta
1,Lucia Quaranta
2,Giacomo Tedesco
3,Carla Longo
4,Carla Longo
5,Ugo Rossi
6,Giuseppe Verdi
7,Maria Rossi
8,Maria Rossi
